# Pre

Import libraries

In [1]:
!pip install wandb
!pip install torch
!pip install transformers
!pip install stopwordsiso

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import json
import nltk
import torch.nn.functional as F
import random,operator
from nltk.tokenize import word_tokenize
from stopwordsiso import stopwords
from nltk.stem.porter import PorterStemmer
import re
import wandb
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
# from torch.utils.data import DataLoader, Dataset
import os, re, string
import torch
import re
import torch.nn as nn
import logging

from transformers import AdamW, get_linear_schedule_with_warmup, AutoTokenizer, AutoModelForSeq2SeqLM, TrainerCallback
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
!nvcc --version

PyTorch version: 2.0.0
CUDA version: 11.8
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [5]:
seed = 42
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:

device = torch.device("cuda:0") if torch.cuda.is_available() else 'cpu'
# device_s = torch.device("cuda:1") if torch.cuda.is_available() else 'cpu'
print("Device: ", device)
!nvidia-smi

Device:  cuda:0
Sun Nov 12 05:30:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                

In [7]:
torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=device, abbreviated=False)

In [8]:
path = '/kaggle/input/data-json/'
save_path = "/kaggle/working/"
checkpoints_path = save_path + 'checkpoints/'

In [9]:
df = pd.read_json(path+"ise-dsc01-train.json")
df = df.transpose()
df.head(10)


,context,claim,verdict,evidence,domain
7125,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",SUPPORTED,"Công ty cũng có chế độ đãi ngộ tốt, có lương t...",giao-duc
7126,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",SUPPORTED,"FABA Technology thành lập năm 2016, chuyên cun...",giao-duc
7127,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",SUPPORTED,"Bên cạnh đó, FABA thường xuyên tổ chức những d...",giao-duc
7128,"Đối với các nhà khoa học, trí thức, điều kiện ...",Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,SUPPORTED,"Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",giao-duc
7129,"Đối với các nhà khoa học, trí thức, điều kiện ...","Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",SUPPORTED,"Trong khi đó, theo báo cáo ""Thị trường nhân lự...",giao-duc
7130,"Đối với các nhà khoa học, trí thức, điều kiện ...",Việc ủng hộ phương án vừa làm việc ở nước ngoà...,SUPPORTED,Tiến sĩ Liêm cũng ủng hộ phương án vừa làm việ...,giao-duc
7131,"Thầy Anh kể, từ nhiều năm trước, thông tin về ...",Một học sinh tâm sự với thầy rằng không dám gó...,SUPPORTED,Học sinh này tâm sự rằng các em rất muốn được ...,giao-duc
7132,"Thầy Anh kể, từ nhiều năm trước, thông tin về ...","Khi tiếp nhận, nếu nội dung phản ánh nằm ngoài...",SUPPORTED,"Nhận tin báo, thầy Anh chuyển nội dung tới côn...",giao-duc
7133,"Thầy Anh kể, từ nhiều năm trước, thông tin về ...","Ví dụ, năm 2020 một học sinh tại THCS Quỳnh Ph...",SUPPORTED,"Hơn hai năm công bố số điện thoại, vị hiệu trư...",giao-duc
7134,"PGS.TS Nguyễn Quang Liệu, hiệu trưởng trường T...",Điểm xét tuyển là tổng điểm các môn thi theo t...,SUPPORTED,Trường không cộng điểm ưu tiên trong tuyển sin...,giao-duc


In [10]:
test_df = pd.read_json(path+"ise-dsc01-private-test-offcial.json")
test_df = test_df.transpose()

test_df.head(10)

,context,claim
39773,"Sáng 7/6, bà Nguyễn Thị Lệ Thanh, Giám đốc Sở ...",Xuyên suốt thời gian diễn ra Festival biển Nha...
37657,"Còn với Công nghệ dệt may, em không biết may h...",Một số người khuyên Nhi Pham theo Công nghệ th...
40265,Nhà nghiên cứu Guangming Tao ở Đại học Khoa họ...,Nhiệt độ da bên dưới lớp cotton cao làm người ...
47711,"Trong không gian của du thuyền 5 sao, Shynh Ho...",Nhãn hàng sẽ cùng 6 đối tác đề ra những chiến ...
39930,Pin sinh học là giải pháp mới để cung cấp năng...,Bụi thông minh chỉ cần 5 tế bào vi khuẩn cũng ...
37208,"Thạc sĩ, bác sĩ Lê Thị Hồng Thắm, khoa Hô hấp ...","Người mắc bệnh tim mạch, thắt cơ tim, dãn tim,..."
38194,Liên kết Dữ liệu An toàn Vaccine thuộc CDC Mỹ ...,CDC chỉ khuyến nghị trẻ em mắc Covid-19 vẫn p...
42930,Sự kiện cũng là không gian để chủ doanh nghiệp...,"Bên cạnh đó, HIU cũng công bố chương trình ""Họ..."
40100,Hãy tưởng tượng thời tiết mùa hè ở bang Louisi...,Mỗi thành viên trong đội tuần tra xoáy nước ấm...
44099,"Theo ông Duy, mức thu 60 triệu đồng mỗi tháng ...",Cảng hoàn thành năm 2015


In [11]:
def clean_text(example):
    text = example
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[ \n]', ' ', text)
    text = re.sub(r' +', ' ', text).strip()
    text = text.lower()
    return text

def remove_stopwords(text):
    stop_words = stopwords(["vi"])
    stop_words_pattern = r'\b(?:{})\b'.format('|'.join(stop_words))
    text = re.sub(stop_words_pattern, '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def preprocess_text(example):
    # Convert to lowercase
    text = clean_text(example)
    
    preprocessed_text = remove_stopwords(text)
    
    return preprocessed_text

In [12]:
verdicts = df['verdict'].unique().tolist()
num_labels = len(verdicts)
def to_category_vector(label):
  return verdicts.index(label)

# Show example
print(to_category_vector("NEI"))
print(num_labels)

1
3


In [13]:
df['evidence'].head(20)

7125    Công ty cũng có chế độ đãi ngộ tốt, có lương t...
7126    FABA Technology thành lập năm 2016, chuyên cun...
7127    Bên cạnh đó, FABA thường xuyên tổ chức những d...
7128    Theo Adeco Việt Nam, mức lương cao nhất đối vớ...
7129    Trong khi đó, theo báo cáo "Thị trường nhân lự...
7130    Tiến sĩ Liêm cũng ủng hộ phương án vừa làm việ...
7131    Học sinh này tâm sự rằng các em rất muốn được ...
7132    Nhận tin báo, thầy Anh chuyển nội dung tới côn...
7133    Hơn hai năm công bố số điện thoại, vị hiệu trư...
7134    Trường không cộng điểm ưu tiên trong tuyển sin...
7135    Điểm xét tuyển vào lớp này là tổng ba môn chun...
7136    Điểm xét tuyển vào lớp này là tổng ba môn chun...
7137    Thí sinh chỉ trúng tuyển một nguyện vọng cao n...
7138    Các ngành thuộc khối sức khỏe có mức điểm chuẩ...
7139    Các ngành thuộc khối sức khỏe có mức điểm chuẩ...
7140     Chỉ tiêu, tổ hợp xét tuyển từng ngành như sau...
7141    Năm ngoái, điểm chuẩn Học viện Ngân hàng theo ...
7142    TS Trầ

In [14]:
df['evidence'] = df['evidence'].fillna('Không có chứng cứ')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37967 entries, 7125 to 35081
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   context   37967 non-null  object
 1   claim     37967 non-null  object
 2   verdict   37967 non-null  object
 3   evidence  37967 non-null  object
 4   domain    37967 non-null  object
dtypes: object(5)
memory usage: 2.7+ MB


In [16]:

# ctxDF = df['context'].apply(text_to_sentences)
# claimDF = df['claim'].apply(text_to_sentences)
ctxDF = df['context'].apply(preprocess_text)
claimDF = df['claim'].apply(preprocess_text)
evidenceDF = df['evidence'].apply(preprocess_text)

testCtxDF = test_df['context'].apply(preprocess_text)
testClaimDF = test_df['claim'].apply(preprocess_text)

In [17]:
df['evidence'] = evidenceDF

In [18]:
# Nối context và claim trong cột mới
df['context_claim'] = "Context: " + ctxDF + " [SEP] Claim: " + claimDF

In [19]:
test_df['context_claim'] = "Context: " + testCtxDF + " [SEP] Claim: " + testClaimDF

In [20]:

print(test_df.at[44099, 'context_claim'])

Context: mức thu 60 triệu đồng bao gồm kho khu vực mặt tiền cảng biển dài 70 m công ty hợp tác đơn vị nâng cấp hướng cảng biển du lịch dân sinh phân lô tiểu thương thuê phản ứng dân đơn vị tạm dừng thu phí tổ chức đối thoại tiểu thương sở hài hoà lợi ích bên lãnh đạo đơn vị chờ cục hàng hải tổ chức đấu giá tìm thầu ba khu vực dân buôn bán đóng phí công ty trúng thầu sách ưu đãi tránh gây sốc dân tài trợ phí neo đậu 700 nghìn đồng 2 triệu đồng trợ giá vé tham quan 3000 đồng giá vé 35000 đồng hỗ trợ chuyến tham quan học tập biểu giá công ty ban hành cano 30 chỗ đóng phí cập bến 100300 nghìn đồng 25 triệu đồng thuê tàu 30 chỗ 300600 nghìn đồng 610 triệu nước dùng rửa tàu sinh hoạt công ty cấp giá 60000 đồng m3 tối thiểu 2 m3 cấp bến cảng 120000 đồng m3 bến phao tối thiểu 15 m3 chủ cano mức phí an thới cảng biển phú quốc nước đầu tư kinh phí 120 tỷ đồng thêm gần 160 tỷ đồng hoàn thiện hạ tầng cảng hoàn thành 2012 công suất thiết kế 280000 hàng hóa 440000 khách cảng phú quốc thể tiếp tàu bi

In [21]:

print(df.at[7125, 'context_claim'])

Context: phát biểu lễ ký kết 1711 giám đốc hành kiêm đồng sáng lập faba technology nguyễn thành hưng định bối cảnh bình hiện mô hình đào tạo 100 online funix cách thể giải toán khan hiếm nhân lực ngành công nghệ thông tin chia sẻ trở thành đối tác đào tạo tuyển dụng funix giúp faba technology đạt mục tiêu trở thành công ty offshore development chất lượng khu vực hợp tác faba kỳ vọng ứng viên chất lượng vị trí công tìm kiếm automation tester uiux designer web developer mobile developer cấp độ tin tưởng chất lượng đào tạo funix nhấn mạnh thu hút giữ chân nhân tài thị trường nhân it khốc liệt faba giới hạn mức lương cập nhật dải lương đạt mức cạnh tranh tốt thị trường công ty chế độ đãi ngộ tốt lương 13 14 phụ cấp bên cạnh faba xuyên tổ chức dự án đào tạo kỹ năng nghề nghiệp kỹ năng mềm trọng xây dựng văn hoá doanh nghiệp thúc đẩy phát triển đội ngũ faba technology thành lập 2016 chuyên cung cấp dịch vụ phát triển phần mềm offshore khách hàng toàn cầu mỹ australia singapore châu âu havas 

In [22]:
verDF = df['verdict'].apply(to_category_vector)
verDF

7125     0
7126     0
7127     0
7128     0
7129     0
        ..
35077    2
35078    2
35079    2
35080    2
35081    2
Name: verdict, Length: 37967, dtype: int64

In [23]:
df['verdict'] = verDF


In [24]:
df

,context,claim,verdict,evidence,domain,context_claim
7125,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Ngoài việc không giới hạn mức lương, công ty c...",0,công ty chế độ đãi ngộ tốt lương 13 14 phụ cấp,giao-duc,Context: phát biểu lễ ký kết 1711 giám đốc hàn...
7126,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Thành lập năm 2016, phát triển phần mềm, tổ ch...",0,faba technology thành lập 2016 chuyên cung cấp...,giao-duc,Context: phát biểu lễ ký kết 1711 giám đốc hàn...
7127,"Phát biểu tại lễ ký kết vào ngày 17/11, Giám đ...","Nhằm phát triển đội ngũ, FABA thường tổ chức n...",0,bên cạnh faba xuyên tổ chức dự án đào tạo kỹ n...,giao-duc,Context: phát biểu lễ ký kết 1711 giám đốc hàn...
7128,"Đối với các nhà khoa học, trí thức, điều kiện ...",Sở dĩ vị trí kỹ sư phần mềm có mức lương cao n...,0,adeco việt nam mức lương đối vị trí kỹ sư phần...,giao-duc,Context: đối khoa học trí thức kiện sở vật chấ...
7129,"Đối với các nhà khoa học, trí thức, điều kiện ...","Theo Adeco Việt Nam, mức lương cao nhất đối vớ...",0,báo cáo thị trường nhân lực ngành công nghệ th...,giao-duc,Context: đối khoa học trí thức kiện sở vật chấ...
...,...,...,...,...,...,...
35077,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...",Thái Lan không dự kiến đón ít nhất 5 triệu khá...,2,thái lan dự kiến đón ít 5 triệu khách trung qu...,the-gioi,Context: cảm sống tự bian hào hứng giấc mơ tru...
35078,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...","Trước đại dịch, người Trung Quốc chiếm gần 1/2...",2,đại dịch trung quốc chiếm gần 13 tổng 40 triệu...,the-gioi,Context: cảm sống tự bian hào hứng giấc mơ tru...
35079,"""Tôi cảm nhận mình đang sống, đang tự do"", Bia...",Zhu nói rằng Số tiền 290.000 USD không thể mua...,2,tiền thể mua hộ nhỏ vị trí đẹp thành phố trung...,the-gioi,Context: cảm sống tự bian hào hứng giấc mơ tru...
35080,"""Tôi không có đầu óc đầu tư chứng khoán và đan...","Tiền tiết kiệm của các hộ gia đình giảm 9,9 ng...",2,tiền tiết kiệm hộ gia đình tăng 99 nghìn tỷ nh...,the-gioi,Context: đầu óc đầu tư chứng khoán chờ điểm hò...


In [25]:
train_contexts_claims = df['context_claim'].values.tolist()
train_verdicts = df['verdict'].values.tolist()
train_evidences = df['evidence'].values.tolist()


test_contexts_claims =test_df['context_claim'].values.tolist()


# Post

In [26]:
# from transformers.models.bart.modeling_bart import BartForConditionalGeneration

# class CustomBartForConditionalGeneration(BartForConditionalGeneration):
#     def shift_tokens_right(self, input_ids, pad_token_id):
#         # Kiểm tra số chiều của input_ids
#         if input_ids.dim() == 1:
#             input_ids = input_ids.unsqueeze(0)

#         # Sử dụng phương pháp tùy chỉnh để shift tokens right
#         shifted_input_ids = input_ids.new_zeros(input_ids.shape)
#         shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
#         shifted_input_ids[:, 0] = pad_token_id

#         return shifted_input_ids

In [27]:

# # Load pre-trained BART model and tokenizer
model_name = 'facebook/bart-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.ERROR)

In [28]:
# class CustomDataset(Dataset):
#     def __init__(self, dataset, tokenizer, max_len_input=512, max_len_output=256):
#         self.dataset = dataset
#         self.tokenizer = tokenizer
#         self.max_len_input = max_len_input
#         self.max_len_output = max_len_output

#     def __getitem__(self, idx):
#         row = self.dataset.iloc[idx]
#         self.tokenizer.padding_side = 'right'

#         input_encoding = self.tokenizer(
#             row['context_claim'],
#             max_length=self.max_len_input,
#             truncation=True,
#             padding="max_length",
#             return_tensors='pt',
#         )
#         item = {key: val.clone().detach()[0] for key, val in input_encoding.items()}

#         # Mã hóa nhãn verdict thành dạng one-hot
#         verdict = row['verdict']  # Lấy giá trị verdict từ dữ liệu
#         labels = F.one_hot(torch.tensor(verdict), num_classes=3).squeeze(0)

#         # Mở rộng labels thành kích thước (10, 512)
# #         labels = labels.unsqueeze(0).expand((10, -1))

#         item['labels'] = labels

#         return item

#     def __len__(self):
#         return self.dataset.shape[0]
# print('done')

In [29]:
# class CustomDataLoader(DataLoader):
#     def __init__(self, dataset, tokenizer, batch_size=1, max_len_input=512, max_len_output=256):
#         self.dataset = dataset
#         self.tokenizer = tokenizer
#         self.max_len_input = max_len_input
#         self.max_len_output = max_len_output
#         self.batch_size = batch_size

#         self.collate_fn = self._collate_fn

#         super().__init__(
#             dataset=dataset,
#             batch_size=batch_size,
#             collate_fn=self.collate_fn
#         )

#     def _collate_fn(self, batch):
#         input_batch = [item['input_ids'] for item in batch]
#         attention_mask_batch = [item['attention_mask'] for item in batch]
#         labels_batch = torch.stack([item['labels'] for item in batch])

#         padded_input_batch = torch.nn.utils.rnn.pad_sequence(input_batch, batch_first=True, padding_value=self.tokenizer.pad_token_id)
#         padded_attention_mask_batch = torch.nn.utils.rnn.pad_sequence(attention_mask_batch, batch_first=True, padding_value=0)

#         return {
#             'input_ids': padded_input_batch,
#             'attention_mask': padded_attention_mask_batch,
#             'labels': labels_batch
#         }
# print('done')

In [30]:
train_labels=train_verdicts
train_labels = torch.tensor(train_labels)
print(train_labels)

tensor([0, 0, 0,  ..., 2, 2, 2])


In [31]:
 train_input_encoding = tokenizer(
            train_contexts_claims,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors='pt',
        )

In [32]:
test_input_encoding = tokenizer(
            test_contexts_claims,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors='pt',
        )

In [33]:
train_input_ids = train_input_encoding.input_ids
train_attention_masks = train_input_encoding.attention_mask

In [34]:

# Tách dữ liệu thành tập huấn luyện và tập validation
train_input_id, val_input_id,train_attention_mask, val_attention_mask, train_input_labels, val_input_labels =train_test_split(train_input_ids,train_attention_masks, train_labels, test_size=0.2, random_state=seed)

In [35]:


# Các tham số huấn luyện
batch_size = 10
epochs = 5
learning_rate = 2e-5
shuffle = True
weight_decay = 0.01
print('done')

done


In [36]:
train_dataset = TensorDataset(train_input_id, train_attention_mask, train_input_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = TensorDataset(val_input_id, val_attention_mask, val_input_labels)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
print('done')

done


In [37]:
# Tạo optimizer và scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=total_steps)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [38]:


# class CustomTrainingCallback(TrainerCallback):
#     def on_train_begin(self, args, state, control, **kwargs):
#         pass

#     def on_train_end(self, args, state, control, **kwargs):
#         pass

#     def on_epoch_begin(self, args, state, control, **kwargs):
#         pass

#     def on_epoch_end(self, args, state, control, **kwargs):
#         pass

#     def on_step_begin(self, args, state, control, **kwargs):
#         pass

#     def on_step_end(self, args, state, control, **kwargs):
#         if control.should_log:
#             optimizer = state.optimizer
#             optimizer.zero_grad()
#             torch.cuda.empty_cache()
# print('done')

In [39]:
wandb.login()
run = wandb.init(
    # Set the project where this run will be logged
    project="UIT_BART_Trainer",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": learning_rate,
        "epochs": epochs,
    })


#a2de7f2aac43b6250bd7d6147079afb0dcf638b5
# !wandb login --relogin

UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])

In [ ]:
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]
#         input_ids = input_ids.unsqueeze(0)
#         attention_mask = attention_mask.unsqueeze(0)
        labels = labels.unsqueeze(1)
#         print(input_ids.shape,attention_mask.shape,labels.shape)
#         labels = labels.unsqueeze(0).expand((10, -1))
        
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
#         print(input_ids.shape)
#         print(attention_mask.shape)
#         print(labels.shape)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
#         loss_fn = nn.CrossEntropyLoss()
#         logits = outputs.logits
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        torch.cuda.empty_cache()
    average_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{epochs} - Average Loss: {average_loss}")
print('done')

In [ ]:

# # Xác định các siêu tham số của Trainer
# training_args = Seq2SeqTrainingArguments(
#     output_dir=checkpoints_path,
#     num_train_epochs=epochs,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     learning_rate=learning_rate,
#     weight_decay=0.01,
#     warmup_steps=500, 
#     evaluation_strategy = "epoch",
#     save_strategy='no',
#     # Các siêu tham số khác
# )
# Tạo optimizer và scheduler
# optimizer = torch.optim.AdamW(model.parameters(), lr=training_args.learning_rate)
# total_steps = len(train_dataset) * training_args.num_train_epochs
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Tạo đối tượng Trainer
# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
#     optimizers=(optimizer, scheduler),
#     callbacks=[CustomTrainingCallback()],

#     # Các đối số khác
# )

# Huấn luyện mô hình
# trainer.train()

In [ ]:

# Save the trained model
# model.save_pretrained('path/to/save/model')
# tokenizer.save_pretrained('path/to/save/tokenizer')

In [ ]:
# # Load pre-trained BART model and tokenizer
# model_name = 'facebook/bart-base'
# model = BartForSequenceClassification.from_pretrained(model_name)
# tokenizer = BartTokenizer.from_pretrained(model_name)

# # Define labels
# labels = ['NEI', 'supported', 'refuted']

# # Preprocess context and claim, and convert to input tensors
# def preprocess_input(context, claim):
#     input_text = f"Context: {context}. Claim: {claim}."
#     input_ids = tokenizer.encode(input_text, add_special_tokens=True)
#     input_tensors = torch.tensor([input_ids])
#     return input_tensors

# # Classify claim using BART model
# def classify_verdict(context, claim):
#     input_tensors = preprocess_input(context, claim)
#     logits = model(input_tensors)[0]
#     predicted_label_idx = torch.argmax(logits, dim=1).item()
#     predicted_label = labels[predicted_label_idx]
#     return predicted_label

# # Example usage
# context = "This is the context."
# claim = "This is the claim."
# predicted_verdict = classify_verdict(context, claim)
# print("Predicted verdict:", predicted_verdict)

# Đánh giá mô hình trên tập validation
# eval_result = trainer.evaluate(eval_dataset=val_dataloader)
# accuracy = eval_result["accuracy"]
# print(f"Accuracy: {accuracy}")

# # Phân loại nhãn cho tập test
# predictions = trainer.predict(test_dataset=test_dataloader)
# # predicted_labels = predictions.predictions.argmax(axis=1)
# import pickle

# # ... Code huấn luyện mô hình ...

# # Lưu lại đối tượng Trainer
# trainer_path = "trainer.pkl"
# with open(trainer_path, "wb") as f:
#     pickle.dump(trainer, f)

# # ... Sau này, khi bạn muốn sử dụng lại Trainer ...

# # Tải lại đối tượng Trainer từ file
# with open(trainer_path, "rb") as f:
#     trainer = pickle.load(f)

# # Sử dụng Trainer đã tải lại
# predictions = trainer.predict(test_dataset=test_dataloader)



# from torch.utils.data import Dataset

# class TestDataset(Dataset):
#     def __init__(self, test_df, tokenizer):
#         self.test_df = test_df
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.test_df)

#     def __getitem__(self, index):
#         context = self.test_df.iloc[index]['context']
#         claim = self.test_df.iloc[index]['claim']
#         inputs = self.tokenizer.encode_plus(
#             context,
#             claim,
#             truncation=True,
#             padding='max_length',
#             max_length=256,
#             return_tensors='pt'
#         )
#         input_ids = inputs['input_ids'].squeeze()
#         attention_mask = inputs['attention_mask'].squeeze()

#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask,
#             'verdict': None  # Trả về None cho nhãn verdict trong tập test
#         }

# test_dataset = TestDataset(test_df, tokenizer)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# predictions = trainer.predict(test_dataset=test_dataloader)
# predicted_labels = predictions.predictions.argmax(axis=1)